In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose


/home/mschulist/anaconda3/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


We will use the inaturalist data set, and try to train an image classifer

In [4]:
training_data = datasets.FashionMNIST(
    root="data",
    transform=ToTensor(),
    download=True,
)

test_data = datasets.FashionMNIST(
    root="data",
    transform=ToTensor(),
    download=True,
)

100%|██████████| 26421880/26421880 [00:02<00:00, 12856419.43it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 276350.69it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4778143.59it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 13287555.07it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [14]:
nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, nn.CrossEntropyLoss(), optimizer)
    test(test_dataloader, model, nn.CrossEntropyLoss())
print("Done!")

Epoch 1
-------------------------------
loss: 2.306082 [    0/60000]
loss: 2.295836 [ 6400/60000]
loss: 2.288682 [12800/60000]
loss: 2.283369 [19200/60000]
loss: 2.270079 [25600/60000]
loss: 2.267504 [32000/60000]
loss: 2.276176 [38400/60000]
loss: 2.260540 [44800/60000]
loss: 2.273446 [51200/60000]
loss: 2.233387 [57600/60000]
Test Error: 
 Accuracy: 32.0%, Avg loss: 0.034982 

Epoch 2
-------------------------------
loss: 2.267155 [    0/60000]
loss: 2.239025 [ 6400/60000]
loss: 2.202436 [12800/60000]
loss: 2.201616 [19200/60000]
loss: 2.178816 [25600/60000]
loss: 2.168465 [32000/60000]
loss: 2.207010 [38400/60000]
loss: 2.162577 [44800/60000]
loss: 2.187305 [51200/60000]
loss: 2.130393 [57600/60000]
Test Error: 
 Accuracy: 32.9%, Avg loss: 0.033205 

Epoch 3
-------------------------------
loss: 2.177827 [    0/60000]
loss: 2.128918 [ 6400/60000]
loss: 2.045554 [12800/60000]
loss: 2.061897 [19200/60000]
loss: 2.032028 [25600/60000]
loss: 2.018627 [32000/60000]
loss: 2.099044 [38400/